<a href="https://colab.research.google.com/github/sakshikorpade/Digital-Communication-System/blob/main/ADC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy import trapz

from numpy import sum,isrealobj,sqrt
from numpy.random import standard_normal
from scipy.special import erfc #erfc/Q function
from itertools import islice

In [ ]:
# Function to plot Graph:

def plot_graph(value):
  
  plt.plot(t,value)
  plt.xlabel("time")
  plt.ylabel("Amplitude")
  plt.show()

In [ ]:

file = open("test.txt","r")
data=file.read()
print(data)
uncompressed_size=len(data)
no_need_chars=[".",",","-","1","8","3","9","5","(",")","\n"]
for i in no_need_chars :
    data = data.replace(i, '')

#data="Sakshi"
compressed_size=len(data)
file.close() 



FileNotFoundError: ignored

In [ ]:
compression_ratio= uncompressed_size/compressed_size
print("uncompressed_size:")
print(uncompressed_size)
print("compressed_size:")
print(compressed_size)
print("compression_ratio:")
print(compression_ratio)

In [ ]:
def source_encoder(data):

   A=" abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ***********"
   B=[[0,0,0,0,0,0],[0,0,0,0,0,1],[0,0,0,0,1,0],[0,0,0,0,1,1],[0,0,0,1,0,0],[0,0,0,1,0,1],[0,0,0,1,1,0],[0,0,0,1,1,1],[0,0,1,0,0,0],[0,0,1,0,0,1],[0,0,1,0,1,0],[0,0,1,0,1,1],[0,0,1,1,0,0],[0,0,1,1,0,1],[0,0,1,1,1,0],[0,0,1,1,1,1],[0,1,0,0,0,0],[0,1,0,0,0,1],[0,1,0,0,1,0],[0,1,0,0,1,1],[0,1,0,1,0,0],[0,1,0,1,0,1],[0,1,0,1,1,0],[0,1,0,1,1,1],[0,1,1,0,0,0],[0,1,1,0,0,1],[0,1,1,0,1,0],[0,1,1,0,1,1],[0,1,1,1,0,0],[0,1,1,1,0,1],[0,1,1,1,1,0],[0,1,1,1,1,1],[1,0,0,0,0,0],[1,0,0,0,0,1],[1,0,0,0,1,0],[1,0,0,0,1,1],[1,0,0,1,0,0],[1,0,0,1,0,1],[1,0,0,1,1,0],[1,0,0,1,1,1],[1,0,1,0,0,0],[1,0,1,0,0,1],[1,0,1,0,1,0],[1,0,1,0,1,1],[1,0,1,1,0,0],[1,0,1,1,0,1],[1,0,1,1,1,0],[1,0,1,1,1,1],[1,1,0,0,0,0],[1,1,0,0,0,1],[1,1,0,0,1,0],[1,1,0,0,1,1],[1,1,0,1,0,0],[1,1,0,1,0,1],[1,1,0,1,1,0],[1,1,0,1,1,1],[1,1,1,0,0,0],[1,1,1,0,0,1],[1,1,1,0,1,0],[1,1,1,0,1,1],[1,1,1,1,0,0],[1,1,1,1,0,1],[1,1,1,1,1,0],[1,1,1,1,1,1]]
   
   encode=np.array([])
   for i in data:
    for j in range(len(A)):
      if i==A[j]:
         en=B[j]
         encode=np.append(encode,en)
   return encode

source_encoded_data=source_encoder(data)
print(source_encoded_data)


In [ ]:
def channel_encoder(source_encoded_data):
    n=3
    channel_encoded_data=np.repeat(source_encoded_data,n)
    return channel_encoded_data

channel_encoded_data=channel_encoder(source_encoded_data)
print(channel_encoded_data)



In [ ]:
n=len(channel_encoded_data)
Tb=400                      # Time duration of each symbol
Eb=0.01                         # Energy per bit (in BPSK energy per symbol)
samples=200

In [ ]:
# Line Coding

def NRZ_polar(data):
  pulse=[]
  for i in data:
    if i==1:
      x=math.sqrt(Eb)
    else:
      x=-math.sqrt(Eb)
    pulse.append(x)
  return pulse

pulse=NRZ_polar(channel_encoded_data)
t=np.linspace(1,n*Tb,samples*len(pulse))
rep_pulse=np.repeat(pulse,samples)
plt.title("NRZ Polar Line Coding")
plot_graph(rep_pulse)


In [ ]:
basis_amp=1/math.sqrt(Tb)
rep_basis=np.repeat(basis_amp,samples*len(pulse))
basis_list=rep_basis.tolist()

def BPSK_Transmitter(rep_basis):

     S=rep_pulse*rep_basis
     return S

S=BPSK_Transmitter(rep_basis)
plt.title("Basis Function")
plot_graph(rep_basis)
plt.title("Transmitted Signal")
plot_graph(S)

In [ ]:
# AWGN channel

def awgn(S,SNRdB):
 SNR_linear = 10**(SNRdB/10) 
 R=[]   
 n=len(channel_encoded_data)
 no=np.array([])
 for i in range(n):
      P=sum((abs(S[i*samples:(i+1)*samples]))**2)*(Tb/samples)
      N0=P/SNR_linear
      X=S[i*samples:(i+1)*samples]
      n = sqrt(N0/2)*standard_normal(X.shape) 
      r=X+n
      R.append(r)
     
 Rnew=np.array(R)
 R_=Rnew.ravel()
 Rn=R_.tolist()
 return Rn

R=awgn(S,0) 
plt.title("Received Signal with noise")   
plot_graph(R)

In [ ]:
def MF_receiver(R,basis_list):
   y_Tb=[]
   decoded_MF=[]
   for i in range(n):

      y=np.convolve(R[i*samples:(i+1)*samples],basis_list[i*samples:(i+1)*samples])
      y_new=y[int(samples)]
      
      y_Tb.append(y_new)

   return y_Tb


output_MF=MF_receiver(R,basis_list)
print(output_MF)




In [ ]:
# ML decoding

def ML_decoder(y_Tb):
   boundary=0
   decoded=[]
   for i in y_Tb:
      if i>boundary:
        d=1
      else:
        d=0
      decoded.append(d)

   received=np.array(decoded)
   
   
   return received
received=ML_decoder(output_MF)
print(received)
print(len(received))

In [ ]:
# BER

def BER(channel_encoded_data,received):
  total=[]
  count=0
  for i in range(0,n):
    if channel_encoded_data[i]==received[i]:
        count=count+0
    else:
        count=count+1
  Pe_ML_detector=count/n
    
  return Pe_ML_detector

Pe_ML_detector=BER(channel_encoded_data,received)
print(Pe_ML_detector)

In [ ]:
def channel_decoder(received):
   element=[]
   for i in range(0,len(received),3):
  
      sum=received[i]+received[i+1]+received[i+2]
      if sum>=2:
        el=1
      else:
        el=0
      element.append(el)
   return element

element=channel_decoder(received)
print(element)



In [ ]:
def source_decoder(element):

   A=" abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ***********"
   B=[[0,0,0,0,0,0],[0,0,0,0,0,1],[0,0,0,0,1,0],[0,0,0,0,1,1],[0,0,0,1,0,0],[0,0,0,1,0,1],[0,0,0,1,1,0],[0,0,0,1,1,1],[0,0,1,0,0,0],[0,0,1,0,0,1],[0,0,1,0,1,0],[0,0,1,0,1,1],[0,0,1,1,0,0],[0,0,1,1,0,1],[0,0,1,1,1,0],[0,0,1,1,1,1],[0,1,0,0,0,0],[0,1,0,0,0,1],[0,1,0,0,1,0],[0,1,0,0,1,1],[0,1,0,1,0,0],[0,1,0,1,0,1],[0,1,0,1,1,0],[0,1,0,1,1,1],[0,1,1,0,0,0],[0,1,1,0,0,1],[0,1,1,0,1,0],[0,1,1,0,1,1],[0,1,1,1,0,0],[0,1,1,1,0,1],[0,1,1,1,1,0],[0,1,1,1,1,1],[1,0,0,0,0,0],[1,0,0,0,0,1],[1,0,0,0,1,0],[1,0,0,0,1,1],[1,0,0,1,0,0],[1,0,0,1,0,1],[1,0,0,1,1,0],[1,0,0,1,1,1],[1,0,1,0,0,0],[1,0,1,0,0,1],[1,0,1,0,1,0],[1,0,1,0,1,1],[1,0,1,1,0,0],[1,0,1,1,0,1],[1,0,1,1,1,0],[1,0,1,1,1,1],[1,1,0,0,0,0],[1,1,0,0,0,1],[1,1,0,0,1,0],[1,1,0,0,1,1],[1,1,0,1,0,0],[1,1,0,1,0,1],[1,1,0,1,1,0],[1,1,0,1,1,1],[1,1,1,0,0,0],[1,1,1,0,0,1],[1,1,1,0,1,0],[1,1,1,0,1,1],[1,1,1,1,0,0],[1,1,1,1,0,1],[1,1,1,1,1,0],[1,1,1,1,1,1]]

   a = np.array(element).reshape(int(len(element)/6),6)
   final=a.tolist()
   decode=np.array([])
   

   for i in final:
      for j in range(len(B)):
         if (i==B[j]):
            de=A[j]
            decode=np.append(decode,de)
            
            #out_arr = np.array_str(decode)
   return decode

decoded_final_data= source_decoder(element)
#print(decoded_final_data)
out=decoded_final_data.tolist()
#print(out)
def listToString(out): 
    str1 = " " 
    return (str1.join(out))
output= listToString(out)
print(output)


In [ ]:
def error_probability(data,decoded_final_data):
    count=0
    for i in range(len(data)):
        if data[i]==decoded_final_data[i]:
           count=count+0
        else:
           count=count+1
    error_prob=count/uncompressed_size
    return error_prob

error_prob= error_probability(data,decoded_final_data)
print(error_prob)

In [ ]:
SNRdB=np.arange(-15,15,1)
BER_=np.array([])
BER_channel_coding=np.array([])

for i in SNRdB:
  
  R=awgn(S,i)
  output_MF=MF_receiver(R,basis_list)
  received=ML_decoder(output_MF)
  Pe_ML_detector=BER(channel_encoded_data,received)
  BER_channel_coding=np.append(BER_channel_coding,Pe_ML_detector)

  element=channel_decoder(received)
  decoded_final_data= source_decoder(element)
  error_prob= error_probability(data,decoded_final_data)
  BER_=np.append(BER_,error_prob)
  
  

fig, ax = plt.subplots(nrows=1,ncols = 1)
ax.semilogy(SNRdB,BER_,color='r',marker='',linestyle='-',label='after channel encoding')
ax.semilogy(SNRdB,BER_channel_coding,color='g',marker='',linestyle='-',label='after BPSK modulation')
ax.legend()
ax.set_xlabel('$E_b/N_0(dB)$');ax.set_ylabel('BER ($P_e$)')
ax.set_title('Probability of Bit Error vs SNR for BPSK over AWGN channel')

In [ ]:
table=np.arange(-15,16,5)
for i in table:
  print("SNR in dB :")
  print(i)
  R=awgn(S,i)
  output_MF=MF_receiver(R,basis_list)
  received=ML_decoder(output_MF)
  Pe_ML_detector=BER(channel_encoded_data,received)
  BER_channel_coding=np.append(BER_channel_coding,Pe_ML_detector)

  element=channel_decoder(received)
  decoded_final_data= source_decoder(element)
  error_prob= error_probability(data,decoded_final_data)
  BER_=np.append(BER_,error_prob)
  print("Error probability:")
  print(error_prob)
  print("\n")
  
